In [6]:
import struct
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import struct
from scipy.stats import norm, kurtosis, stats

from helpFunctions import *

#----------
path = "C:\\Kandidat\\R&D\\Generalized-Deduplication-in-Python\\Bearing Data"
path_npz = "C:\\Kandidat\\R&D\\Generalized-Deduplication-in-Python\\CompressedData"

np.set_printoptions(suppress=True)          # Suppress scientific notation

In [7]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
print(len(onlyfiles))
dataa = list()
for index in onlyfiles[0:3]:                        #Currently loads 3 data chunks aka 1.5million values
    header, data = loadData(os.path.join(path, index))
    dataa.extend(data)
    print(index)
d_og = dataa.copy()

20
hoco-dek3kas1-m2_sensor2_2020-07-10_07-24-21.npz
hoco-dek3kas1-m2_sensor2_2020-07-10_08-24-43.npz
hoco-dek3kas1-m2_sensor2_2020-07-10_09-25-09.npz
1500000
0.0190719
<class 'list'>
0.0190719
1500000
<class 'numpy.ndarray'>
{'machine_id': 'hoco-dek3kas1-m2', 'sensor_id': 'sensor2', 'timestamp': 1594373109370, 'sensor_type': ' VSA', 'rpm': 2198.69, 'sample_frequency': 50000.0}


In [9]:
print(type(d_og[1499999]))
print(len(d_og))
#del listsofBases, listsofCounts, listsofData, data, dataa, header, index, onlyfiles, testere

<class 'numpy.float64'>
1500000


In [11]:
listsofData = list()
listsofBases = list()
listsofCounts = list()
listsofData, listsofBases, listsofCounts = Compressor_Meta_lists(d_og)

LOOP :0


KeyboardInterrupt: 

In [17]:
for Saver in range(0,len(listsofBases)):
    if os.path.exists(os.path.join(path_npz,'Deviation_bits'+str(Saver)+'.npz')) == False:
        file_name = 'Deviation_bits'+str(Saver)
        np.savez( os.path.join(path_npz, file_name), Compressed_Data=listsofData[Saver], Bases = listsofBases[Saver], Counts = listsofCounts[Saver] )
        print('Deviation_bits '+str(Saver)+' Created<<') 
    else:
        pass
        

In [ ]:
def Compressor_Meta_lists(f):                              # Compress data and return 0-32 devations
    results_Data        = list()
    # results_Deviations  = list()
    results_Bases       = list()
    results_Counts      = list()
    temp_data           = list()
    temp_base           = list()
    temp_counts         = list()
    for idx in range(0,33):
        temp_data.clear(); temp_base.clear(); temp_counts.clear(); baseCounter = 0
        print("LOOP :"+str(idx))

        for c in range(0,len(f)):
            binary = float_to_bin(f[c])
            if idx == 0:
                base = binary[:-1-idx] + "0"                                # if deviation bits 0 dont multiply by 0
            else:
                base = binary[:0-idx] + "0" * idx                           # Assemple new stored value
            #tempFlip = binary[-1:-index-1:(-1)]                            # Read thru bits from behind, used to gather deviation bits
            #cDeviation = tempFlip[::-1]                                    # Flip found deviation bits, ready to be concatanated to cBase
            temp_data.append(np.float64(bin_to_float(base)))
            if base not in temp_base:
                temp_base.append(base)
                temp_counts.append(["ID: "+str(baseCounter), 1, "deviations bits : "+str(idx)])
                baseCounter += 1
            else:
                temp_counts[temp_base.index(base)][1] += 1
            if c == len(f)-1:
                results_Data.append(temp_data[:])
                results_Bases.append(temp_base[:])
                results_Counts.append(temp_counts[:])
                print("SAVE")

    return results_Data, results_Bases, results_Counts